In [1]:
%matplotlib qt5

In [2]:
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
from mathphys.functions import save_pickle, load_pickle

In [28]:
def set_hchmb(acc, strin, height):
    indices = pyaccel.lattice.find_indices(acc, 'fam_name', strin)
    for iten in indices:
        acc[iten].hmax = height
        acc[iten].hmin = -height
    
    return None

# this function returns to nominal value for vacum chamber height, if it is necessary at some point

def nom_hchmb(acc, strin): 
    model = pymodels.si.create_accelerator()
    hmax = pyaccel.lattice.get_attribute(model, 'hmax', indices='open')
    hmin = pyaccel.lattice.get_attribute(model, 'hmin', indices='open')
    indices = pyaccel.lattice.find_indices(model, 'fam_name', strin)
    for iten in indices:
        acc[iten].hmax = hmax[iten]
        acc[iten].hmin = hmin[iten]
    
    return None

def track_mchn_stdy(acc, bunch, param, increment, index, nturn):
    # setting param to be the index of the array [x,x',y,y',delta,z]

    bunch[param] += increment
    tracked = pyaccel.tracking.ring_pass(acc, particles=bunch,
                                         nr_turns=nturn, turn_by_turn= True,
                                         element_offset= index, parallel=True)
    
    part_out, lost_flag, turn_lost, index_lost, plane_lost = tracked
    
    turnl_element = []

    for i,iten in enumerate(turn_lost):
        if iten == nturn and index_lost[i] == index: # ignora elétrons que não foram perdidos
            pass
        else:
            turnl_element.append((iten, index_lost[i]))
    
    return turnl_element

def varying_incmnts(acc,bunch,par,rng, index, nturn, val, strin, height):

    ''' 
        acc: accelerator 
        bunch: particle initial conditions
        par: index of [x,x',y,y',delta,z] -> [0,1,2,3,4,5]
        rng: range of the increment
        nturn: number of turns
        val: variable that goes in numpy linspace, val is the value till the variation must be performed
        strin: is a string, this string represents the fam_name of the scraper 'SHVC' or 'SVVC'
        height: is the value that the user desires for setting the height of vchamber
    '''

    lostpos_n = []
    lostpos_m = []
    losttur_n = []
    losttur_m = []

    inc = np.linspace(1e-4, val, rng)
    incdif = np.diff(inc)

    inc_qlst_n = []
    inc_qlst_m = []

    # nominal model tracking simulation
    bunchi = bunch.copy()
    for j, iten in enumerate(incdif):
        track = track_mchn_stdy(acc, bunchi, par, increment=iten, index=index, nturn=nturn)
        length = len(track)
        if len(track) == 0:
            pass
        else:
            inc_qlst_n.append([inc[j+1],length]) # this will be the y axis of the graphic
            for lst in track:
                lost_turn, lost_pos = lst
                losttur_n.append([lost_turn, inc[j+1]])
                lostpos_n.append([lost_pos, inc[j+1]])

    set_hchmb(acc, strin, height) # after calling this function, vchamber's height will be changed

    # # changed height model tracking simulation
    bunchi = bunch.copy()
    for j, iten in enumerate(incdif):
        track = track_mchn_stdy(acc, bunchi, par, increment=iten, index=index, nturn=nturn)
        length = len(track)
        if len(track) == 0:
            pass
        else:
            inc_qlst_m.append([inc[j+1],length]) # this will be the y axis of the graphic
            for lst in track:
                lost_turn, lost_pos = lst
                losttur_m.append([lost_turn, inc[j+1]])
                lostpos_m.append([lost_pos, inc[j+1]])

    # return lostpos_n, lostpos_m

    return lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m



def p_sim(lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m, spos):
    fig1, (a1n,a2n,a3n) = plt.subplots(nrows=1,ncols=3, sharey=True, figsize=(10,5))

    fig1.suptitle('tracking without scraper')

    for iten in inc_qlst_n:
        a1n.plot(iten[1], iten[0] , '.', color='blue')

    for iten in losttur_n:
        a2n.plot(iten[0], iten[1], '.', color='blue')

    for iten in lostpos_n:
        a3n.plot(spos[iten[0]], iten[1], '.', color='blue')

    fig2, (a1m,a2m,a3m) = plt.subplots(nrows=1,ncols=3, sharey=True, figsize=(10,5))

    fig2.suptitle('tracking with scraper')

    for iten in inc_qlst_m:
        a1m.plot(iten[1], iten[0] , '.', color='blue')

    for iten in losttur_m:
        a2m.plot(iten[0], iten[1], '.', color='blue')

    for iten in lostpos_m:
        a3m.plot(spos[iten[0]], iten[1], '.', color='blue')

    plt.show()


In [4]:
fitm = pymodels.si.create_accelerator()
fitm = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fitm)
eqparams = pyaccel.optics.beam_envelope.EqParamsFromBeamEnvelope(fitm)

# parameters to generate the beam
h_emitt = eqparams.emit1
v_emitt = h_emitt/100
sigmae = eqparams.espread0
bun_len = eqparams.bunlen

# defining optcis
twiss,*_ = pyaccel.optics.calc_twiss(fitm, indices='closed')

# defining the index 
dicty = pyaccel.lattice.find_dict(fitm, 'fam_name')
nlk_index = dicty['InjNLKckr'][0] + 1
spos = pyaccel.lattice.find_spos(fitm, indices='open')

# generating gaussian beam.
bunch = pyaccel.tracking.generate_bunch(50, envelope=None, emit1=h_emitt,
                                emit2=v_emitt, sigmae= sigmae, sigmas=bun_len, optics=twiss[nlk_index])


In [5]:
# acc,bunch,par,rng, index, nturn, val, strin, height parameters that will be usefull


In [31]:
variable = varying_incmnts(fitm, bunch[:,0:20], 0, 100, nlk_index, 7, 0.02,'SHVC', 0.001)
lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m = variable

In [32]:
p_sim(lostpos_n, lostpos_m, losttur_n, losttur_m, inc_qlst_n, inc_qlst_m, spos)

In [ ]:
# c = 93
# indices = c_data_treat(variable, c, 0.01,'x')
# plot_tracked(indices, spos, indices, 'x',c, 0.01,100)

In [ ]:
# # anotações que eu pensei que pudessem ser importantes

# # definindo também o modelo fitado para calcular a distribuição de equilíbrio do feixe
# # eu não sei exatamente se essa celula no meu código é util, porque em um primeiro momento vamos variar apenas
# # as posições x, y, x' e y'

# fit_acc = pymodels.si.create_accelerator()
# fit_acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fit_acc)
# tousan = tousclass.Tous_analysis(fit_acc)
# deltas = tousan.deltas
# n_turns = tousan.nturns

# orb = pyaccel.tracking.find_orbit6(acc, indices=[0, nlk_index])
# orb = orb[:, 1]
# np.zeros((6, deltas.size)), orb[:, None]

# rin = np.zeros((6, deltas.size))
# rin += orb[:,None]

# mean = np.zeros(6)
# env = pyop.beam_envelope.calc_beamenvelope(fit_acc)
# cov = env[nlk_index]
# particles = np.random.multivariate_normal(mean, cov, size=1).T

# # célula colocada aqui simplesmente para teste de verificação
# # ao executar o código abaixo é possível observar a distribuição gaussiana das posições em x
# plt.figure()
# plt.hist(particles[0], bins=200)
# plt.show()

# # esse procedimento é feito para ir variando os parâmetros de posição 
# vecy = np.linspace(4.8332669321064365e-09, 4.8332669321064365e-02, 1000)
# vecx = np.linspace(2.25702434e-08,2.25702434e-02, 1000)
# # com a função vstack do numpy podemos juntar dois arrays e formar uma matriz
# np.vstack((vecx, vecy))

In [ ]:
# def s_data_treat(l_of_tples, idx, val):
    
#     size = len(l_of_tples)
#     inc = np.linspace(0,val, size)
#     for c, iten in enumerate(l_of_tples):
#         if len(iten) == 0:
#             pass
#         else:
#             j = c
#             break

#     if len(l_of_tples[idx]) == 0:
#         print("all the electrons survived")
#         return

#     else:
#         nt_lost, indices = np.zeros(len(l_of_tples[idx])), np.zeros(len(l_of_tples[idx]))

#         for j,iten in enumerate(l_of_tples[idx]):
#             nt_lostj, idx_lost = iten
#             nt_lost[j] = nt_lostj
#             indices[j] = idx_lost
#         indices = np.intp(indices)
#     ind = np.arange(indices.size) # this index corresponds to how many times the variation of the parameter has been performed

#     return c, inc[c], ind, nt_lost, indices

# def c_data_treat(tup, idx, val, par):

#     '''
#     tup: this tuple contains two lists, these lists are the tracking simulation without and with varying the height of the
#     vchamber from a marker named as horizontal scraper or vertical scraper
#     idx: the variation's index of the parameter 
#     val: variable that goes in numpy linspace, val is the value till the variation must be performed
#     '''

#     all_data = []
#     for iten in tup:
#         resul_tup = s_data_treat(iten, idx, val)

#         c, inc_c, ind, nt_lost, indices = resul_tup
#         all_data.append(indices)
#         print('começa a haver perda de elétrons a partir do incremento {} em {}'.format(inc_c, par))

#     return all_data

# # para montar a função que vai realizar o plot dos gráficos podemos simplesmente passar a função acima em um for

# def plot_tracked(acc, spos, l_all_data, par, idx, val, total): # é bom que essa função receba os valores das lista de indices onde as paticulas foram perdidas

#     '''
#     acc: accelerator
#     spos: s position array along the ring
#     l_all_data: list containing the data that will be analised 
#     par: is a string, corresponding to the variation of the parameter, that will be shown in the title of the graphic/ x, x'
#     idx: is the value that counts wich index was chosen to be analised
#     val: variable that goes in numpy linspace, val is the value till the variation must be performed
#     '''
    
#     nom_track = l_all_data[0] # keep in mind that nominal_tracking is the list of indices that the electrons were lost in the nominal_model with vchamber's height defined by default
#     mod_track = l_all_data[1]
#     inc = np.linspace(0,val, total)

#     ind_nom = np.arange(nom_track.size)
#     ind_mod = np.arange(mod_track.size)

#     fig, (a1, a2) = plt.subplots(nrows= 1, ncols= 2, figsize=(10,5), sharey=True)

#     a1.set_title(r'Without scrapers, varying {}'.format(par), fontsize=16)
#     a2.set_title(r'With scrapers, varying {}'.format(par), fontsize=16)

#     a1.set_xlabel(r's position [m]', fontsize=14)
#     a1.set_ylabel(r'# lost electron', fontsize=14)
    
#     a2.set_xlabel(r's position [m]', fontsize=14)
#     a2.set_ylabel(r'# lost electron', fontsize=14)

#     a1.scatter(spos[nom_track],ind_nom, s=4, label='Value of increment {:.4f} in {}'.format(inc[idx], par))
#     # pyaccel.graphics.draw_lattice(acc, offset=0, height=0, gca=True)
#     a1.legend()
#     print(nom_track)
#     a2.scatter(spos[mod_track],ind_mod, s=4, label='Value of increment {:.4f} in {}'.format(inc[idx], par))
#     a2.legend()
#     # pyaccel.graphics.draw_lattice(acc, offset=0, height=0, gca=True)
#     plt.show()

#     return None